In [ ]:
import pandas as pd
frex_by_topic = pd.read_csv("C:\\Users\\arnea\\OneDrive\\Desktop\\Thesis\\Work\\Python\\top_fre_words_per_topic.csv")

frex_by_topic = frex_by_topic.rename(columns={'Word': 'word_stemmed'})
frex_by_topic = frex_by_topic.rename(columns={'Topic': 'topic'})
frex_by_topic = frex_by_topic.rename(columns={'Probability': 'frex'})

total_words = frex_by_topic.shape[0]
print("\nNumber of words:", total_words)

unique_topics_count = frex_by_topic['topic'].nunique()
print("Number of unique topics:", unique_topics_count)
print("Words per topic:", 5100 / 17)

frex_by_topic.head()

In [ ]:
# Load and merge to have the stemmed version of the words
dictionary_stem = pd.read_csv("CC:\\Users\\arnea\\OneDrive\\Desktop\\Thesis\\Work\\Python\\dictionary_stem.csv") 

# Calculate the total number of different articles (rows) as total
total_words = dictionary_stem.shape[0]
print("\nNumber of words:", total_words)

dictionary_stem.head()

# merge the two dataframes
df = pd.merge(dictionary_stem, frex_by_topic, on='word_stemmed', how='right')

total_words = df.shape[0]
print("\nNumber of words:", total_words)

df.head()

In [ ]:
#The below is not very good. Use this tutorial instead, maybe with an external RA.
#https://diging.github.io/tethne/doc/0.6.1-beta/tutorial.mallet.html
#To calculate the 5 strongest connections between topics based on the shared first 30 words, we need to sort the edges by their weight (number of shared words) and select the top 5 connections.

import networkx as nx
import matplotlib.pyplot as plt

# Assuming 'frex_by_topic' DataFrame has 'topic' and 'word_stemmed' columns

# Group the words by topic into a dictionary
groups = frex_by_topic.groupby('topic')['word_stemmed'].apply(list).to_dict()

# Create an empty graph
G = nx.Graph()

# Add nodes (topics) to the graph
G.add_nodes_from(groups.keys())

# Add edges between topics that share common words (only considering the first 300 words for each topic)
for group1, words1 in groups.items():
    for group2, words2 in groups.items():
        if group1 != group2:
            shared_words = set(words1[:300]).intersection(words2[:300])  # Consider only the first 30 words
            if shared_words:
                G.add_edge(group1, group2, weight=len(shared_words))

# Sort edges by weight (number of shared words) in descending order
sorted_edges = sorted(G.edges(data=True), key=lambda x: x[2]['weight'], reverse=True)

# Assign colors to nodes based on topics
# You can define your color mapping for different topics here
# For simplicity, let's generate random colors for each topic
import random
group_colors = {topic: "#" + ''.join([random.choice('0123456789ABCDEF') for j in range(6)]) for topic in G.nodes()}

# Get node colors based on topics
node_colors = [group_colors[topic] for topic in G.nodes()]

# Draw the graph with reduced font size for labels
plt.figure(figsize=(12, 10))
pos = nx.spring_layout(G, seed=42)  # You can use different layouts
nx.draw(G, pos, with_labels=True, node_color=node_colors, node_size=500, font_weight='bold', font_size=8)  # Adjust font_size here

# Highlight the top 5 strongest connections
for edge in strongest_connections:
    nx.draw_networkx_edges(G, pos, edgelist=[(edge[0], edge[1])], width=2, edge_color='red')

# Show the plot
plt.title('Topic Correlation based on shared FREX words (first 30 words). In red the 4 strongest connections.')
plt.show()
# Sort edges by weight (number of shared words) in descending order
sorted_edges = sorted(G.edges(data=True), key=lambda x: x[2]['weight'], reverse=True)

# Get the top 5 strongest connections
strongest_connections = sorted_edges[:10]

# Create a list to store the strongest connection pairs and their strength
strongest_connection_list = []

# Print the top 10 strongest connections and store them in the list
print("Top 10 Strongest Connections Pairs:")
for edge in strongest_connections:
    strength = edge[2]['weight']
    connection_pair = (edge[0], edge[1], strength)
    strongest_connection_list.append(connection_pair)
    print(edge[:2], "Strength:", strength)
    

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

# Initializing VADER SentimentIntensityAnalyzer
sia = SIA()

# Calculating sentiment for each word in the 'term' column
results_word_sentiment = []
for idx, row in df.iterrows():
    word = str(row['word'])  # Convert 'word' to string explicitly
    pol_score = sia.polarity_scores(word)
    pol_score['word'] = word
    pol_score['word_stemmed'] = row['word_stemmed']
    pol_score['frex'] = row['frex']
    pol_score['topic'] = row['topic']
    results_word_sentiment.append(pol_score)
    
# Creating a DataFrame from the sentiment scores of each word
word_sentiment_scores = pd.DataFrame(results_word_sentiment)

# Displaying the sentiment scores for each word
print(word_sentiment_scores)

In [ ]:
# KDE plot for compound sentiment scores
sns.kdeplot(data=word_sentiment_scores['compound'], fill=True, color='blue', label='Compound')
# KDE plot for neg sentiment scores
sns.kdeplot(data=word_sentiment_scores['neg'], fill=True, color='orange', label='Neg')


plt.title('Kernel Density Estimate of Sentiment Scores')
plt.xlabel('Sentiment Scores')
plt.ylabel('Density')
plt.legend()  # Show legend with labels for compound and neg
plt.show()

# Multiply 'frex' with 'neg', 'neu', 'pos', 'compound' columns
word_sentiment_scores['neg_weighted'] = word_sentiment_scores['frex'] * word_sentiment_scores['neg']
word_sentiment_scores['neu_weighted'] = word_sentiment_scores['frex'] * word_sentiment_scores['neu']
word_sentiment_scores['pos_weighted'] = word_sentiment_scores['frex'] * word_sentiment_scores['pos']
word_sentiment_scores['compound_weighted'] = word_sentiment_scores['frex'] * word_sentiment_scores['compound']

# Replace 0 values with 0 in the new weighted columns
columns_to_adjust = ['neg_weighted', 'neu_weighted', 'pos_weighted', 'compound_weighted']
word_sentiment_scores[columns_to_adjust] = word_sentiment_scores[columns_to_adjust].where(word_sentiment_scores[columns_to_adjust] != 0, 0)

# Display the updated DataFrame
print(word_sentiment_scores.head())

# Selecting specific columns
selected_columns = ['word', 'neg', 'neu', 'pos']
selected_data = word_sentiment_scores[selected_columns]

# Export the selected columns to an Excel file
#selected_data.to_excel('frex_word_sentiment_scores.xlsx', index=False)

# Displaying a confirmation message
print("Export to 'frex_word_sentiment_scores.xlsx' completed successfully.")

# Grouping by 'topic' and topic_labels, aggregating the weighted columns by sum
aggregate_sentiment_by_topic = word_sentiment_scores.groupby('topic')[['neg_weighted', 'neu_weighted', 'pos_weighted', 'compound_weighted']].sum()

# Displaying the aggregate sentiment scores by 'topic'
print(aggregate_sentiment_by_topic)

In [ ]:
#Normalize the 3 different variables 

from sklearn.preprocessing import MinMaxScaler

# Initialize the MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1, 1))

# Columns to normalize
columns_to_normalize = ['compound_weighted', 'neg_weighted', 'pos_weighted']

# Fit and transform the data to normalize it between -1 and 1 for each column
normalized_values = scaler.fit_transform(aggregate_sentiment_by_topic[columns_to_normalize])

# Create a DataFrame with the normalized values
normalized_df = pd.DataFrame(normalized_values, columns=columns_to_normalize, index=aggregate_sentiment_by_topic.index)

# Update the original DataFrame with normalized values
aggregate_sentiment_by_topic[columns_to_normalize] = normalized_df

print(aggregate_sentiment_by_topic)

# This is the original plot, however I manually changed the colors such that values <0.2 are yellow now

import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

# Sort the 'topic_labels' based on 'compound_weighted' values in descending order
sorted_topic_labels = aggregate_sentiment_by_topic.sort_values(by='compound_weighted', ascending=True).index

# Reorder 'compound_weighted' values based on the sorted labels
sorted_compound_weighted_values = aggregate_sentiment_by_topic.loc[sorted_topic_labels, 'compound_weighted']

# Define colormap and normalize your data to map it to colors
cmap = plt.get_cmap('RdYlGn')
norm = Normalize(vmin=min(sorted_compound_weighted_values), vmax=max(sorted_compound_weighted_values))

# Calculate colors based on normalized values
colors = [cmap(norm(value)) for value in sorted_compound_weighted_values]

# Create a horizontal bar chart with colored bars
plt.figure(figsize=(10, 6))
bars = plt.barh(sorted_topic_labels, sorted_compound_weighted_values, color=colors)

# Set plot title and labels
plt.title("Topic sentiment using the FREX-approach (300 words/topic)")
plt.xlabel("Aggregate weighted compound sentiment, normalized and sorted in descending order")
plt.ylabel("Topic")

# Create a colorbar to indicate values
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])  
cbar = plt.colorbar(sm, ax=plt.gca())  # Explicitly specify the axes for colorbar

# Show the plot
plt.show()

In [ ]:
# Set up a colormap with 15 distinct colors
colors = plt.cm.tab20.colors  # You can use other colormaps as per your preference

# Create an array of marker styles for distinguishing between different topics
markers = ['o', 's', 'D', '^', 'v', '<', '>', 'p', 'P', '*', 'h', 'H', '+', 'x', '|']

# Set up the plot
plt.figure(figsize=(8, 6))

# Iterate over each topic label (which is assumed to be the index)
for i, (index, row) in enumerate(aggregate_sentiment_by_topic.iterrows()):
    topic_label = index
    plt.scatter(row['pos_weighted'], row['neg_weighted'], label=topic_label, s=100,
                color=colors[i], marker=markers[i % len(markers)])

# Set plot limits and labels
plt.xlim(-1.1, 1.1)
plt.ylim(-1.1, 1.1)
plt.xlabel('Weighted positive sentiment')
plt.ylabel('Weighted negative sentiment')

# Show a legend with the topic labels
plt.legend(title='Topic', bbox_to_anchor=(1.05, 1), loc='upper left')

# Set plot title
plt.title('Scatterplot for negative and positive sentiments by topic')

# Show the plot
plt.grid(True)
plt.show()